##    第1章　API周りの初期設定
---



**はじめに[Kaggle](https://www.kaggle.com/)のアカウントからAPIを使うためにjson持ってきて入れましょう**

In [0]:
from google.colab import files
files.upload()

{}

**jsonを入れれたらcolab上で使えるように権限を変えましょう**

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls -l /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

## 第2章　ライブラリの追加

---



**今回はNNの定義にkerasを使うのでKerasを入れます。後は予測データを直接kaggleにアップするのでkaggleも入れます。**

In [0]:
!pip install -q keras
!pip install -q kaggle


 68% 50.0M/73.2M [00:00<00:00, 84.2MB/s]
100% 73.2M/73.2M [00:00<00:00, 166MB/s] 
 86% 42.0M/48.8M [00:00<00:00, 79.4MB/s]
100% 48.8M/48.8M [00:00<00:00, 150MB/s] 
  0% 0.00/235k [00:00<?, ?B/s]
100% 235k/235k [00:00<00:00, 25.1MB/s]


**これでKaggleのAPIが使えるようになったので、早速 train/test データをダウンロードしましょう**

In [0]:
!kaggle competitions download -c digit-recognizer

## 第3章　実際にやってみる
---

**最初にライブラリをインポートしましょう**

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# keras
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.optimizers import SGD, RMSprop
from keras.callbacks import ReduceLROnPlateau  
from sklearn.model_selection import train_test_split

**データを読み込んで正規化、ラベル等の前処理を行います。**

In [0]:
# load training & test datasets
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

# pandas to numpy
Y_train = train["label"]
X_train = train.drop(labels=["label"], axis=1)

del train

# normalize
X_train = X_train/255.0
test = test/255.0

# reshape the data so that the data
# represents (label, img_rows, img_cols, grayscale)
X_train = X_train.values.reshape(-1, 28, 28, 1)
test = test.values.reshape(-1, 28, 28, 1)

# one-hot vector as a label
Y_train = to_categorical(Y_train, num_classes=10)

**CNNの層構成を定義します。**

In [0]:
# set the CNN No.1
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(28,28,1)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(10))

model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_5 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
activation_6 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
batch_normalization_6 (Batch (None, 24, 24, 32)        128       
_________________________________________________________________
activation_7 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
__________

**先ほどのモデルを使用して、10クラス分類を行う設定にします。また、画像拡張を行います。**

In [0]:
# compile model
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# cross validation
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=1220)

# learning rate
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', patience=3, verbose=1, factor=0.5, min_lr=0.00001)

# data argumentation
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.08, zoom_range=0.08)
train_generator = gen.flow(X_train, Y_train, batch_size=64)

**実際に回します。**

In [0]:
# model training
model.fit_generator(train_generator, epochs=30, validation_data = (X_val, Y_val), verbose=2, steps_per_epoch=X_train.shape[0]/36,
                                       callbacks=[learning_rate_reduction])

Epoch 1/50
 - 30s - loss: 0.1146 - acc: 0.9640 - val_loss: 0.0671 - val_acc: 0.9825
Epoch 2/50
 - 28s - loss: 0.0477 - acc: 0.9850 - val_loss: 0.0450 - val_acc: 0.9881
Epoch 3/50
 - 28s - loss: 0.0410 - acc: 0.9876 - val_loss: 0.0746 - val_acc: 0.9852
Epoch 4/50
 - 28s - loss: 0.0331 - acc: 0.9899 - val_loss: 0.0321 - val_acc: 0.9923
Epoch 5/50
 - 28s - loss: 0.0329 - acc: 0.9898 - val_loss: 0.0317 - val_acc: 0.9892
Epoch 6/50
 - 28s - loss: 0.0285 - acc: 0.9918 - val_loss: 0.0385 - val_acc: 0.9915
Epoch 7/50
 - 28s - loss: 0.0253 - acc: 0.9928 - val_loss: 0.0177 - val_acc: 0.9952
Epoch 8/50
 - 28s - loss: 0.0267 - acc: 0.9924 - val_loss: 0.0344 - val_acc: 0.9921
Epoch 9/50
 - 28s - loss: 0.0240 - acc: 0.9931 - val_loss: 0.0288 - val_acc: 0.9944
Epoch 10/50
 - 28s - loss: 0.0220 - acc: 0.9935 - val_loss: 0.0378 - val_acc: 0.9934

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 11/50
 - 28s - loss: 0.0155 - acc: 0.9955 - val_loss: 0.0292 - val_acc: 

## 第4章　提出してみる


---


**学習したモデルでtestデータの予測をします。**

In [0]:
# model prediction on test data
predictions = model.predict_classes(test, verbose=0)

**predictionをcsvに変換して提出、完了です。**

In [0]:
# make a submission file
submissions = pd.DataFrame({"ImageId": list(range(1,len(predictions)+1)), "Label": predictions})
submissions.to_csv("my_submission.csv", index=False, header=True)

# submit the file to kaggle
!kaggle competitions submit digit-recognizer -f my_submission.csv -m "ok Google Colab!"

Successfully submitted to Digit Recognizer

### 番外編です。GPUではなく、TPUを使ったモデルでやってみます。
---

**※TPUを使用する場合です。TensorflowのバックエンドとしてKerasを使うので、ライブラリを再び設定します。**

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.contrib.tpu.python.tpu import keras_support
# from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, AveragePooling2D, Dense, Dropout, Flatten, MaxPooling2D
# from tensorflow.keras.models import Model, Sequential
# from tensorflow.keras.utils import to_categorical


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from tensorflow.keras.layers import ThresholdedReLU 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau  
from sklearn.model_selection import train_test_split

**少しネットワーク構成を変えます**

In [0]:
# Set the CNN model No2

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 7, 7, 64)          0         
__________

**先ほどのモデルを使用して、10クラス分類を行う設定にします。また、画像拡張を行います。**

In [0]:
# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

# Compile the model
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

# Set a learning rate annealer
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(X_train)

**実際に回します。**

In [0]:
# Fit the model
history = model.fit_generator(datagen.flow(X_train,Y_train, batch_size=64), epochs = 30, validation_data = (X_val,Y_val),
                              verbose = 2, steps_per_epoch=X_train.shape[0], callbacks=[learning_rate_reduction])